Install Libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# # cell1 : install libraries.
# # !pip install torch torchvision torchaudio
# # !pip install albumentations opencv-python matplotlib numpy pandas pycocotools
# # !pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
# # cell 2 : set dataset paths
# import os
# from glob import glob

# train_img_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/train/images"
# train_ann_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/train/Annotations"
# val_img_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/val/images"
# val_ann_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/val/Annotations"

# assert os.path.exists(train_img_dir)
# assert os.path.exists(train_ann_dir)
# assert os.path.exists(val_img_dir)
# assert os.path.exists(val_ann_dir)

# print("Dataset paths verified successfully!")
# # cell 3: custom clahe transform
# import cv2
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
# from albumentations.core.transforms_interface import ImageOnlyTransform

# class CLAHETransform(ImageOnlyTransform):
#     def __init__(self, clip_limit=2.0, tile_grid_size=(8,8), always_apply=False, p=1.0):
#         super(CLAHETransform, self).__init__(always_apply=always_apply, p=p)
#         self.clip_limit = clip_limit
#         self.tile_grid_size = tile_grid_size

#     def apply(self, image, **params):
#         lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
#         l, a, b = cv2.split(lab)
#         clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
#         cl = clahe.apply(l)
#         merged = cv2.merge((cl, a, b))
#         return cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)

#     def get_transform_init_args_names(self):
#         return ("clip_limit", "tile_grid_size")
# # cell 4 : transforms
# def get_train_transform():
#     return A.Compose([
#         CLAHETransform(),
#         A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
#         A.HueSaturationValue(20, 30, 20, p=0.5),
#         A.RGBShift(15, 15, 15, p=0.5),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
#         A.Resize(640, 640),
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

# def get_val_transform():
#     return A.Compose([
#         CLAHETransform(),
#         A.Resize(640, 640),
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
# # cell 5: dataset class
# import torch
# from torch.utils.data import Dataset
# import xml.etree.ElementTree as ET
# import numpy as np
# from PIL import Image

# class RSODDataset(Dataset):
#     def __init__(self, image_dir, annotation_dir, transforms=None):
#         self.image_dir = image_dir
#         self.annotation_dir = annotation_dir
#         self.transforms = transforms
#         self.image_files = sorted(glob(os.path.join(image_dir, "*.jpg")) + glob(os.path.join(image_dir, "*.png")))
#         self.class_dict = {"aircraft": 0, "oiltank": 1, "overpass": 2, "playground": 3}

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_path = self.image_files[idx]
#         image = np.array(Image.open(img_path).convert("RGB"))
#         xml_path = os.path.join(self.annotation_dir, os.path.basename(img_path).replace(".jpg", ".xml").replace(".png", ".xml"))
#         tree = ET.parse(xml_path)
#         root = tree.getroot()

#         boxes, labels = [], []
#         for obj in root.findall('object'):
#             class_name = obj.find('name').text.lower()
#             if class_name not in self.class_dict:
#                 continue
#             bbox = obj.find('bndbox')
#             xmin = float(bbox.find('xmin').text)
#             ymin = float(bbox.find('ymin').text)
#             xmax = float(bbox.find('xmax').text)
#             ymax = float(bbox.find('ymax').text)
#             boxes.append([xmin, ymin, xmax, ymax])
#             labels.append(self.class_dict[class_name])

#         boxes = np.array(boxes, dtype=np.float32) if boxes else np.zeros((0, 4))
#         labels = np.array(labels) if labels else np.zeros(0,)

#         if self.transforms:
#             transformed = self.transforms(image=image, bboxes=boxes, labels=labels)
#             image = transformed["image"]
#             boxes = torch.as_tensor(transformed["bboxes"], dtype=torch.float32)
#             labels = torch.as_tensor(transformed["labels"], dtype=torch.int64)

#         return image, {"boxes": boxes, "labels": labels}

# train_dataset = RSODDataset(train_img_dir, train_ann_dir, get_train_transform())
# val_dataset = RSODDataset(val_img_dir, val_ann_dir, get_val_transform())

# print(f"Training samples: {len(train_dataset)}")
# print(f"Validation samples: {len(val_dataset)}")
# # cell 6 : meodel setup
# import torchvision
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
# from torchvision.ops import MultiScaleRoIAlign

# def get_model(num_classes):
#     backbone = resnet_fpn_backbone('resnet101', pretrained=True, trainable_layers=5)
#     anchor_generator = AnchorGenerator(sizes=((32,), (64,), (128,), (256,), (512,)),
#                                        aspect_ratios=((0.5, 1.0, 2.0, 3.0),)*5)
#     roi_pooler = MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'], output_size=7, sampling_ratio=2)
#     model = FasterRCNN(backbone, num_classes=num_classes,
#                        rpn_anchor_generator=anchor_generator,
#                        box_roi_pool=roi_pooler,
#                        box_score_thresh=0.05)
#     return model

# model = get_model(num_classes=5)
# # cell 7 : dataloader and optimizer
# from torch.utils.data import DataLoader

# def collate_fn(batch):
#     return tuple(zip(*batch))

# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn)

# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# # cell 8 : training + evaluation with focal loss
# import time
# from torch.nn import functional as F

# class FocalLoss(torch.nn.Module):
#     def __init__(self, alpha=0.25, gamma=2, reduction='mean'):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.reduction = reduction

#     def forward(self, inputs, targets):
#         ce_loss = F.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-ce_loss)
#         loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
#         return loss.mean() if self.reduction == 'mean' else loss.sum()

# def train_one_epoch(model, optimizer, data_loader, device, epoch):
#     model.train()
#     total_loss = 0
#     focal_loss = FocalLoss()

#     for images, targets in data_loader:
#         images = [img.to(device).float() for img in images]
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#         loss_dict = model(images, targets)

#         losses = sum(loss for loss in loss_dict.values())
#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()

#         total_loss += losses.item()

#     return total_loss / len(data_loader)

# def evaluate(model, data_loader, device):
#     model.eval()
#     eval_loss = 0

#     with torch.no_grad():
#         for images, targets in data_loader:
#             images = [img.to(device) for img in images]
#             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#             # Use model in training mode temporarily to compute loss
#             model.train()
#             loss_dict = model(images, targets)
#             model.eval()

#             losses = sum(loss for loss in loss_dict.values())
#             eval_loss += losses.item()

#     return eval_loss / len(data_loader)


# num_epochs = 20
# for epoch in range(num_epochs):
#     start = time.time()
#     train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
#     val_loss = evaluate(model, val_loader, device)
#     scheduler.step()
#     print(f"Epoch {epoch+1}/{num_epochs} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | Time: {time.time()-start:.2f}s")
# # cell 9 : coco evaluation
# from pycocotools.coco import COCO
# from pycocotools.cocoeval import COCOeval
# import torch
# import json

# def evaluate_coco(model, data_loader, device):
#     model.eval()
#     results = []
#     image_ids = []

#     with torch.no_grad():  # Important: disable gradients for inference
#         for i, (images, targets) in enumerate(data_loader):
#             images = [img.to(device) for img in images]
#             outputs = model(images)

#             for j, output in enumerate(outputs):
#                 image_id = targets[j].get("image_id", torch.tensor(i * len(images) + j)).item()
#                 boxes = output["boxes"].detach().cpu().numpy()   # ✅ detach() here
#                 scores = output["scores"].detach().cpu().numpy() # ✅ detach()
#                 labels = output["labels"].detach().cpu().numpy() # ✅ detach()

#                 for k in range(len(boxes)):
#                     result = {
#                         "image_id": image_id,
#                         "category_id": int(labels[k]),
#                         "bbox": [
#                             float(boxes[k][0]),
#                             float(boxes[k][1]),
#                             float(boxes[k][2] - boxes[k][0]),
#                             float(boxes[k][3] - boxes[k][1])
#                         ],
#                         "score": float(scores[k])
#                     }
#                     results.append(result)

#     with open("predictions.json", "w") as f:
#         json.dump(results, f)

#     # Build fake COCO-style GT from val_dataset
#     val_image_ids = list(range(len(val_dataset)))
#     coco_gt = COCO()
#     dataset = {
#         "info": {
#             "description": "RSOD Validation",
#             "version": "1.0",
#             "year": 2025
#         },
#         "licenses": [],
#         "images": [{"id": i} for i in val_image_ids],
#         "categories": [
#             {"id": 0, "name": "aircraft"},
#             {"id": 1, "name": "oiltank"},
#             {"id": 2, "name": "overpass"},
#             {"id": 3, "name": "playground"}
#         ],
#         "annotations": []
#     }


#     ann_id = 0
#     for img_id in val_image_ids:
#         _, target = val_dataset[img_id]
#         boxes = target["boxes"].detach().numpy()   # ✅ detach() here too
#         labels = target["labels"].detach().numpy()

#         for k in range(len(boxes)):
#             annotation = {
#                 "id": ann_id,
#                 "image_id": img_id,
#                 "category_id": int(labels[k]),
#                 "bbox": [
#                     float(boxes[k][0]),
#                     float(boxes[k][1]),
#                     float(boxes[k][2] - boxes[k][0]),
#                     float(boxes[k][3] - boxes[k][1])
#                 ],
#                 "area": float((boxes[k][2] - boxes[k][0]) * (boxes[k][3] - boxes[k][1])),
#                 "iscrowd": 0
#             }
#             dataset["annotations"].append(annotation)
#             ann_id += 1

#     coco_gt.dataset = dataset
#     coco_gt.createIndex()

#     coco_dt = coco_gt.loadRes("predictions.json")
#     coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
#     coco_eval.evaluate()
#     coco_eval.accumulate()
#     coco_eval.summarize()

#     class_aps = {}
#     class_names = ["aircraft", "oiltank", "overpass", "playground"]

#     for i, class_name in enumerate(class_names):
#         coco_eval.params.catIds = [i]
#         coco_eval.evaluate()
#         coco_eval.accumulate()
#         coco_eval.summarize()
#         class_aps[class_name] = coco_eval.stats[1]  # AP@0.5 for this class

#     return coco_eval.stats[0], class_aps  # mAP@0.5, per-class AP
# # cell 10 : run final evaluation
# mAP, class_aps = evaluate_coco(model, val_loader, device)

# print("\nFinal Evaluation Results:")
# print(f"mAP@0.5: {mAP:.4f}")
# print("Per-class AP@0.5:")
# for class_name, ap in class_aps.items():
#     print(f"{class_name}: {ap:.4f}")


📁 Set Dataset Paths

In [6]:
# cell 2 : set dataset paths
import os
from glob import glob

train_img_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/train/images"
train_ann_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/train/Annotations"
val_img_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/val/images"
val_ann_dir = "/content/drive/MyDrive/csv/RSOD_YOLO/val/Annotations"

assert os.path.exists(train_img_dir)
assert os.path.exists(train_ann_dir)
assert os.path.exists(val_img_dir)
assert os.path.exists(val_ann_dir)

print("Dataset paths verified successfully!")


Dataset paths verified successfully!


🔄 Custom CLAHE Transform

In [7]:
# cell 3: custom clahe transform
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

class CLAHETransform(ImageOnlyTransform):
    def __init__(self, clip_limit=2.0, tile_grid_size=(8,8), always_apply=False, p=1.0):
        super(CLAHETransform, self).__init__(always_apply=always_apply, p=p)
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def apply(self, image, **params):
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        cl = clahe.apply(l)
        merged = cv2.merge((cl, a, b))
        return cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)

    def get_transform_init_args_names(self):
        return ("clip_limit", "tile_grid_size")


🔁 Transforms

In [8]:
# cell 4 : transforms
def get_train_transform():
    return A.Compose([
        CLAHETransform(),
        A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
        A.HueSaturationValue(20, 30, 20, p=0.5),
        A.RGBShift(15, 15, 15, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Resize(640, 640),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

def get_val_transform():
    return A.Compose([
        CLAHETransform(),
        A.Resize(640, 640),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


📦 Dataset Class

In [9]:
# cell 5: dataset class
import torch
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET
import numpy as np
from PIL import Image

class RSODDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transforms=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transforms = transforms
        self.image_files = sorted(glob(os.path.join(image_dir, "*.jpg")) + glob(os.path.join(image_dir, "*.png")))
        self.class_dict = {"aircraft": 1, "oiltank": 2, "overpass": 3, "playground": 4}

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = np.array(Image.open(img_path).convert("RGB"))
        xml_path = os.path.join(self.annotation_dir, os.path.basename(img_path).replace(".jpg", ".xml").replace(".png", ".xml"))
        tree = ET.parse(xml_path)
        root = tree.getroot()

        boxes, labels = [], []
        for obj in root.findall('object'):
            class_name = obj.find('name').text.lower()
            if class_name not in self.class_dict:
                continue
            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(self.class_dict[class_name])

        boxes = np.array(boxes, dtype=np.float32) if boxes else np.zeros((0, 4))
        labels = np.array(labels) if labels else np.zeros(0,)

        if self.transforms:
            transformed = self.transforms(image=image, bboxes=boxes, labels=labels)
            image = transformed["image"]
            boxes = torch.as_tensor(transformed["bboxes"], dtype=torch.float32)
            labels = torch.as_tensor(transformed["labels"], dtype=torch.int64)

        return image, {"boxes": boxes, "labels": labels}

train_dataset = RSODDataset(train_img_dir, train_ann_dir, get_train_transform())
val_dataset = RSODDataset(val_img_dir, val_ann_dir, get_val_transform())

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Training samples: 749
Validation samples: 187


/tmp/ipython-input-7-1982176329.py:9: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super(CLAHETransform, self).__init__(always_apply=always_apply, p=p)


🧠 Model Setup

In [10]:
# cell 6 : meodel setup
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign

def get_model(num_classes):
    backbone = resnet_fpn_backbone('resnet101', pretrained=True, trainable_layers=5)
    anchor_generator = AnchorGenerator(sizes=((32,), (64,), (128,), (256,), (512,)),
                                       aspect_ratios=((0.5, 1.0, 2.0, 3.0),)*5)
    roi_pooler = MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'], output_size=7, sampling_ratio=2)
    model = FasterRCNN(backbone, num_classes=num_classes,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler,
                       box_score_thresh=0.05)
    return model

model = get_model(num_classes=5)


🔄 DataLoader and Optimizer

In [11]:
# cell 7 : dataloader and optimizer
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

🏋️ Training + Evaluation with Focal Loss

In [13]:
# cell 8 : training + evaluation with focal loss
import time
from torch.nn import functional as F

class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return loss.mean() if self.reduction == 'mean' else loss.sum()

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0
    focal_loss = FocalLoss()

    for images, targets in data_loader:
        images = [img.to(device).float() for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    eval_loss = 0

    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Use model in training mode temporarily to compute loss
            model.train()
            loss_dict = model(images, targets)
            model.eval()

            losses = sum(loss for loss in loss_dict.values())
            eval_loss += losses.item()

    return eval_loss / len(data_loader)


num_epochs = 3
for epoch in range(num_epochs):
    start = time.time()
    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    val_loss = evaluate(model, val_loader, device)
    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | Time: {time.time()-start:.2f}s")


Epoch 1/3 | Train: 0.8704 | Val: 0.7166 | Time: 309.95s
Epoch 2/3 | Train: 0.6948 | Val: 0.6135 | Time: 252.15s
Epoch 3/3 | Train: 0.5835 | Val: 0.5045 | Time: 251.71s


📊 COCO Evaluation

In [17]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json

def evaluate_coco(model, data_loader, device):
    model.eval()
    results = []

    with torch.no_grad():
        for i, (images, targets) in enumerate(data_loader):
            images = [img.to(device) for img in images]
            outputs = model(images)

            for j, output in enumerate(outputs):
                image_id = targets[j].get("image_id", torch.tensor(i * len(images) + j)).item()
                boxes = output["boxes"].detach().cpu().numpy()
                scores = output["scores"].detach().cpu().numpy()
                labels = output["labels"].detach().cpu().numpy()

                for k in range(len(boxes)):
                    result = {
                        "image_id": image_id,
                        "category_id": int(labels[k]),  # your labels are 1-indexed!
                        "bbox": [
                            float(boxes[k][0]),
                            float(boxes[k][1]),
                            float(boxes[k][2] - boxes[k][0]),
                            float(boxes[k][3] - boxes[k][1])
                        ],
                        "score": float(scores[k])
                    }
                    results.append(result)

    with open("predictions.json", "w") as f:
        json.dump(results, f)

    val_image_ids = list(range(len(val_dataset)))
    coco_gt = COCO()
    dataset = {
        "info": {"description": "RSOD Validation", "version": "1.0", "year": 2025},
        "licenses": [],
        "images": [{"id": i} for i in val_image_ids],
        "categories": [
            {"id": 1, "name": "aircraft"},
            {"id": 2, "name": "oiltank"},
            {"id": 3, "name": "overpass"},
            {"id": 4, "name": "playground"}
        ],
        "annotations": []
    }

    ann_id = 0
    for img_id in val_image_ids:
        _, target = val_dataset[img_id]
        boxes = target["boxes"].detach().numpy()
        labels = target["labels"].detach().numpy()

        for k in range(len(boxes)):
            annotation = {
                "id": ann_id,
                "image_id": img_id,
                "category_id": int(labels[k]),  # make sure it's 1-indexed to match!
                "bbox": [
                    float(boxes[k][0]),
                    float(boxes[k][1]),
                    float(boxes[k][2] - boxes[k][0]),
                    float(boxes[k][3] - boxes[k][1])
                ],
                "area": float((boxes[k][2] - boxes[k][0]) * (boxes[k][3] - boxes[k][1])),
                "iscrowd": 0
            }
            dataset["annotations"].append(annotation)
            ann_id += 1

    coco_gt.dataset = dataset
    coco_gt.createIndex()

    coco_dt = coco_gt.loadRes("predictions.json")
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    class_aps = {}
    class_names = ["aircraft", "oiltank", "overpass", "playground"]

    for i, class_name in enumerate(class_names, start=1):  # start=1 matches your label IDs!
        coco_eval.params.catIds = [i]
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        class_aps[class_name] = coco_eval.stats[1]  # AP@0.5 for this class

    return coco_eval.stats[0], class_aps  # mAP@0.5, per-class AP


 Run Final Evaluation

In [18]:
# cell 10 : run final evaluation
mAP, class_aps = evaluate_coco(model, val_loader, device)

print("\nFinal Evaluation Results:")
print(f"mAP@0.5: {mAP:.4f}")
print("Per-class AP@0.5:")
for class_name, ap in class_aps.items():
    print(f"{class_name}: {ap:.4f}")


creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.82s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.386
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.798
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.412
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.570
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.190
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.520
 Average Recall     (AR) @[ IoU=0.5

In [16]:
model.eval()
images, targets = next(iter(val_loader))
images = [img.to(device) for img in images]

with torch.no_grad():
    outputs = model(images)

print("Predicted labels:", [output["labels"] for output in outputs])


Predicted labels: [tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1,
        3, 4, 1, 1], device='cuda:0'), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3,
        3, 3, 1, 1, 1, 4, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1], device='cuda:0')]
